# IPU RAY LIBRARY LITERATE TEST

This notebook contains instructions to configure, build, run, and test a [Poplar C++ Ray/Path Tracer](https://github.com/markp-gc/ipu_ray_lib) for Graphcore IPUs. This serves to both test the application and to document how test/debug.

## Testing Method

There are automated tests and interactive ones. We will run the interactive ones first, then the automated tests. We recommend you click "run all" in the notebook to execute everything and then read through to understand whether everything is working as expected.

### Build the Code

This notebook assumes you are starting from a clean checkout. The following cell configures and builds everything. The build uses CMake:

In [ ]:
!mkdir -p build
%cd build
!cmake -Wno-dev -G Ninja ..
!ninja -j64



### Run the Test

First check we can run the application by rendering a path traced image of the Cornell Box:

In [ ]:
!./trace -w 720 -h 720 --render-mode path-trace --visualise rgb --samples 1000 --ipus 4 --ipu-only --box-only

The output image is high dynamic range (HDR) in EXR format. We can make a function
to perform a quick tone-mapping and display the resulting image in Python:

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

# Function to apply simple gamma correction, rescale,
# and clip values into range 0-255:
def gamma_correct(x, exposure, gamma):
  scale = 2.0 ** exposure
  y = np.power(x * scale, 1.0 / gamma) * 255.0
  return np.clip(y, 0.0, 255.0)

# Function to plot an opencv image:
def display_image(img):
  plt.figure(figsize=(6, 6))
  plt.style.use('dark_background')
  plt.imshow(cv2.cvtColor(ldr, cv2.COLOR_BGR2RGB), interpolation='bicubic')
  plt.show()

EXR_FLAGS = cv2.IMREAD_UNCHANGED | cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH
hdr = cv2.imread('out_rgb_ipu.exr', EXR_FLAGS)
print(f"HDR image shape: {hdr.shape} type: {hdr.dtype} min: {np.min(hdr)} max: {np.max(hdr)}")

ldr = gamma_correct(hdr, exposure=1.2, gamma=2.4).astype(np.uint8)
cv2.imwrite('tonemapped.png', ldr)
display_image(ldr)

If you want to render a CPU reference image remove the option `--ipu-only` but be aware it will take
much much longer to render. (For a list of all command options run `./test --help`.)

## Comparing AOVs with Embree

We can compare arbitrary output variables (AOVs) to the same scene rendered with Embree. This demonstrates the basic ray trace functionality works and is also useful for debugging when things are broken. For this we use a quicker render-mode `shadow-trce`. E.g. to compare normals with Embree:

In [ ]:
!./trace -w 1440 -h 1440 --render-mode shadow-trace --visualise normal --ipus 4

Once the outputs are ready we can load them into Python to compare:

In [ ]:
# Load normals:
ipu_normals = cv2.imread('out_normal_ipu.exr', EXR_FLAGS)
cpu_normals = cv2.imread('out_normal_cpu.exr', EXR_FLAGS)
embree_normals = cv2.imread('out_normal_embree.exr', EXR_FLAGS)

compare = ipu_normals
abs_err = np.abs(compare - embree_normals)
print(f"IPU normals min: {np.min(compare)} max: {np.max(compare)}")
print(f"Embree normals min: {np.min(embree_normals)} max: {np.max(embree_normals)}")
print(f"ABS Error min: {np.min(abs_err)} max: {np.max(abs_err)} mean: {np.mean(abs_err)}")

# Plot them side by side:
vis = ((compare + 1.0) / 2.0)
vis_embree = ((embree_normals + 1.0) / 2.0)
fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(12, 6))
ax[0].imshow(vis)
ax[0].set_title('IPU')
ax[1].imshow(vis_embree)
ax[1].set_title('Embree')
plt.show()

We can plot an error histogram (using a log scale because the error counts are small). As you can see most errors are tiny but there are a few outliers - these will be rays that hit alternative (i.e. possibly valid within machine precision) objects due to differences between our intersection test code and Embree's:

In [ ]:
plt.hist(abs_err.flatten(), bins=300, range=[0.0, np.max(abs_err)], log=True)
plt.show()

## Automated Tests

Finally we can check the automated tests:

In [ ]:
!./tests